Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.  
Mobile Games AB Testing with Cookie Cats  
https://www.kaggle.com/code/mursideyarkin/mobile-games-ab-testing-with-cookie-cats/input

In [2]:
import numpy as np
import pandas as pd
# установки бутсрапа нет т.к. уже установлен.
import bootstrapped.bootstrap as bs
import bootstrapped.compare_functions as bs_compare
import bootstrapped.stats_functions as bs_stats


In [3]:
df = pd.read_csv('результаты А_B для 9 ДЗ - cookie_cats 2.csv')
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,0,0
1,337,gate_30,38,1,0
2,377,gate_40,165,1,0
3,483,gate_40,1,0,0
4,488,gate_40,179,1,1


In [4]:
df.userid.duplicated().sum()  # дубликаты userid


0

In [5]:
df.isnull().sum() # пропущенные данные

userid            0
version           0
sum_gamerounds    0
retention_1       0
retention_7       0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  int64 
 4   retention_7     90189 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 3.4+ MB


Отлично! Задублированных нету. Пропущенных нету. Тип данных устраивает.

In [7]:
df["userid"].nunique()  # количество игроков

90189

In [8]:
# количество игроков при этапе 30
df[df["version"] == "gate_30"]["userid"].nunique()

44700

In [9]:
# количество игроков при этапе 40
df[df["version"] == "gate_40"]["userid"].nunique()

45489

Количество игроков в группах достаточно и группы примерно одинаковы. Разделим на две группы, что можно было сразу сделать...

In [10]:
df_gate_30 = df[df["version"] == "gate_30"]
df_gate_30.head

<bound method NDFrame.head of         userid  version  sum_gamerounds  retention_1  retention_7
0          116  gate_30               3            0            0
1          337  gate_30              38            1            0
6         1066  gate_30               0            0            0
11        2101  gate_30               0            0            0
13        2179  gate_30              39            1            0
...        ...      ...             ...          ...          ...
90179  9998576  gate_30              14            1            0
90180  9998623  gate_30               7            0            0
90182  9999178  gate_30              21            1            0
90183  9999349  gate_30              10            0            0
90186  9999710  gate_30              28            1            0

[44700 rows x 5 columns]>

In [11]:
df_gate_40 = df[df["version"] == "gate_40"]
df_gate_40.head

<bound method NDFrame.head of         userid  version  sum_gamerounds  retention_1  retention_7
2          377  gate_40             165            1            0
3          483  gate_40               1            0            0
4          488  gate_40             179            1            1
5          540  gate_40             187            1            1
7         1444  gate_40               2            0            0
...        ...      ...             ...          ...          ...
90181  9998733  gate_40              10            1            0
90184  9999441  gate_40              97            1            0
90185  9999479  gate_40              30            0            0
90187  9999768  gate_40              51            1            0
90188  9999861  gate_40              16            0            0

[45489 rows x 5 columns]>

In [12]:
ret_30_1 = df_gate_30['retention_1'].mean()
ret_30_7 = df_gate_30['retention_7'].mean()
ret_40_1 = df_gate_40['retention_1'].mean()
ret_40_7 = df_gate_40['retention_7'].mean()
ret_30_1, ret_30_7, ret_40_1, ret_40_7 # решил всё в одном блоке запихать т.к. лень разносить и достаточно проверить что посчиталось.

(0.4481879194630872,
 0.19020134228187918,
 0.44228274967574577,
 0.18200004396667327)

In [13]:
b = bs.bootstrap_ab(df_gate_30['retention_1'].values, df_gate_40['retention_1'].values, stat_func=bs_stats.mean,
                    compare_func=bs_compare.difference, alpha=0.05, num_iterations=1000)

print(b.lower_bound, b.upper_bound)

-0.0008114792779020073 0.012283089310919728


"Нулик попал" :) Или ДИ пересекает О, статистически значимых различий нет. 

In [14]:
b = bs.bootstrap_ab(df_gate_30['retention_7'].values, df_gate_40['retention_7'].values, stat_func=bs_stats.mean,
                    compare_func=bs_compare.difference, alpha=0.05, num_iterations=1000)

print(b.lower_bound, b.upper_bound)


0.002923098563101226 0.013185973421408257


"Нулик промазал" :) Или ДИ не пересекает О, статистически значимые различия есть. 

Вывод: В первый день игроки почти не столкнулись с разницей где находится этап. Потому ожидаемо почти одинаковый ретеншен. Но на 7 день ретеншен лучше когда этап находится на 30-м уровне, чем на 40-м. Потому рекомендуется оставить этап на 30 уровне. НО не помешало бы проанализировать внутреигровые покупки т.к. покупок может быть больше.